In [ ]:
class ExtraS1S2Properties(hax.minitrees.TreeMaker):
    '''
    
    '''
    __version__ = '0.0.0'
    extra_branches = ['peaks.*', 'interactions*']
    
    
    def extract_data(self, event):
        # ================ PEAK PROPERTIES =========================
        # Extract data for these types
        peak_types = ['s1', 's2']
        
        # Peak_properties (will be loaded for s1_0, s1_1, s2_0, s2_1)
        peak_properties = ['n_saturated_channels']

        result_vars = [_pt + '_' + _pp for _pt in peak_types for for _pp in peak_properties ]
        result = {k: float('nan') for k in result_vars}
        
        s1s = []
        s2s = []
        n_interactions = len(event.interactions)
        
        # For the first two interactions, take their s1 and s2 index, if they exist
        for interaction in event.interactions()
            # Get s1 and s2 index
            s1s.append(interaction.s1)
            s2s.append(interaction.s2)

        
        # ================= LOAD PEAK DATA ==========================
        # Loop over the two peak types
        for pt in peak_types:
            # Load index list
            if pt == 's1':
                peak_list = s1s
            elif pt == 's2':
                peak_list = s2s
            else:
                raise NotImplementedError('Peak types s1 and s2 supported only!')
            
            # Now, for this peak type, loop over peak_indices and get peak pos
            if len(peak_list) == 0:
                # If there are not enough peaks of this type, do not try to find properties
                continue
            else:
                peak_pos = int(peak_list[pi])

                # Time to extract data
                peak_name = pt + ('_%d' % pi) + '_' # Example: s1_0_

                # All standard properties
                for prop in peak_properties:
                    result[peak_name + prop] = getattr(event.peaks[peak_pos], prop)
        
        return result